### **块 1: 环境设置、库导入与路径管理**

**目标:** 初始化项目环境，加载所有必需的库，并根据 `RUNNING_ENV` 和 `TEST_MODE` 智能配置所有输入输出文件的路径。此代码块负责读取人工审核的最终成果，并为大规模应用这些规则到新闻语料库上做好准备。

In [1]:
# =============================================================================
# --- 块 1: 环境设置、库导入与路径管理 ---
# =============================================================================

# 作用: 导入所有项目运行所需的Python库。
import pandas as pd
import os
import pickle
import time
from tqdm.auto import tqdm
import psutil
import shutil
import ahocorasick
import gc

# --- 核心配置区 ---
# 作用: 全局控制参数，方便调试与切换运行模式。
RUNNING_ENV = 'local'
TEST_MODE = False
TEST_SAMPLE_SIZE = 1000

# --- 并行处理配置 ---
# 作用: 智能检测CPU核心数，并为多进程处理设定合理的进程数。
cpu_cores = psutil.cpu_count(logical=True)
N_PROCESSES = min(cpu_cores - 1 if cpu_cores > 1 else 1, 8)
if N_PROCESSES < 1: N_PROCESSES = 1

# --- 路径智能管理 ---
# 作用: 根据运行环境（本地或服务器）自动构建正确的文件路径。
print(f"检测到运行环境为: 【{RUNNING_ENV.upper()}】")
TEMP_DIR = '/tmp'

if RUNNING_ENV == 'local':
    print("使用 'local' 模式的相对路径。")
    BASE_DATA_PROCESSED_PATH = '../data_processed'
elif RUNNING_ENV == 'dsw':
    print("使用 'dsw' 模式的绝对路径。")
    BASE_DATA_PROCESSED_PATH = '/mnt/data/data_processed'
    if not os.path.exists(TEMP_DIR): os.makedirs(TEMP_DIR)
else:
    raise ValueError(f"未知的 RUNNING_ENV: '{RUNNING_ENV}'. 请选择 'local' 或 'dsw'。")

# 作用: 定义所有输入输出文件的“原始”存储位置。
# 输入文件
FINAL_CHINA_NEWS_ORIGINAL = os.path.join(BASE_DATA_PROCESSED_PATH, 'final_china_news.csv')
REVIEWED_CANDIDATES_ORIGINAL = os.path.join(BASE_DATA_PROCESSED_PATH, 'candidate_phrases_for_review_reviewed.csv')
EXPERT_RULES_ORIGINAL = os.path.join(BASE_DATA_PROCESSED_PATH, 'expert_rules.csv')
# 输出文件
MERGE_DICT_ORIGINAL = os.path.join(BASE_DATA_PROCESSED_PATH, 'merge_dict.pkl')
NEW_STOPWORDS_ORIGINAL = os.path.join(BASE_DATA_PROCESSED_PATH, 'new_stopwords.pkl')
SOLIDIFIED_TEXT_ORIGINAL = os.path.join(BASE_DATA_PROCESSED_PATH, 'china_news_solidified.pkl')

# 作用: 初始化代码块实际使用的路径变量，默认为原始路径。
FINAL_CHINA_NEWS_PATH = FINAL_CHINA_NEWS_ORIGINAL
REVIEWED_CANDIDATES_PATH = REVIEWED_CANDIDATES_ORIGINAL
EXPERT_RULES_PATH = EXPERT_RULES_ORIGINAL
MERGE_DICT_PATH = MERGE_DICT_ORIGINAL
NEW_STOPWORDS_PATH = NEW_STOPWORDS_ORIGINAL
SOLIDIFIED_TEXT_PATH = SOLIDIFIED_TEXT_ORIGINAL

# 作用: 在服务器环境下，智能地将高I/O负载的文件复制到本地临时目录进行操作，以提升速度和稳定性。
if RUNNING_ENV == 'dsw':
    print("DSW 环境模式已激活，将智能检查并使用本地临时目录 /tmp ...")

    def sync_to_tmp_if_needed(original_path, temp_dir):
        if not os.path.exists(original_path):
            raise FileNotFoundError(f"关键输入文件在源路径不存在: {original_path}")
        filename = os.path.basename(original_path)
        temp_path = os.path.join(temp_dir, filename)
        if not os.path.exists(temp_path) or os.path.getsize(original_path) != os.path.getsize(temp_path):
            print(f"正在从 {original_path} 同步到 {temp_path}...")
            shutil.copy(original_path, temp_path)
            print("同步完成。")
        else:
            print(f"临时文件 {temp_path} 已是最新，跳过同步。")
        return temp_path

    try:
        # 作用: 重定向路径变量，使后续代码透明地使用临时目录中的文件。
        FINAL_CHINA_NEWS_PATH = sync_to_tmp_if_needed(FINAL_CHINA_NEWS_ORIGINAL, TEMP_DIR)
        REVIEWED_CANDIDATES_PATH = sync_to_tmp_if_needed(REVIEWED_CANDIDATES_ORIGINAL, TEMP_DIR)
        EXPERT_RULES_PATH = sync_to_tmp_if_needed(EXPERT_RULES_ORIGINAL, TEMP_DIR)

        MERGE_DICT_PATH = os.path.join(TEMP_DIR, 'merge_dict.pkl')
        NEW_STOPWORDS_PATH = os.path.join(TEMP_DIR, 'new_stopwords.pkl')
        SOLIDIFIED_TEXT_PATH = os.path.join(TEMP_DIR, 'china_news_solidified.pkl')
    except FileNotFoundError as e:
        print(f"❌ 致命错误: {e}")
        raise e

# 作用: 在程序开始时清晰地展示所有最终配置，便于检查和追溯。
print("\n--- 环境准备 ---")
if TEST_MODE:
    print(f"🚀🚀🚀 运行在【快速测试模式】下，将处理前 {TEST_SAMPLE_SIZE} 行新闻！🚀🚀🚀")
else:
    print(f"🚢🚢🚢 运行在【完整数据模式】下，将处理所有新闻。🚢🚢🚢")
print(f"新闻语料输入: {FINAL_CHINA_NEWS_PATH}")
print(f"审核候选输入: {REVIEWED_CANDIDATES_PATH}")
print(f"专家规则输入: {EXPERT_RULES_PATH}")
print(f"合并词典输出: {MERGE_DICT_PATH}")
print(f"停用词输出: {NEW_STOPWORDS_PATH}")
print(f"固化文本输出: {SOLIDIFIED_TEXT_PATH}")
print(f"将使用 {N_PROCESSES} 个进程进行并行处理。")

检测到运行环境为: 【LOCAL】
使用 'local' 模式的相对路径。

--- 环境准备 ---
🚢🚢🚢 运行在【完整数据模式】下，将处理所有新闻。🚢🚢🚢
新闻语料输入: ../data_processed\final_china_news.csv
审核候选输入: ../data_processed\candidate_phrases_for_review_reviewed.csv
专家规则输入: ../data_processed\expert_rules.csv
合并词典输出: ../data_processed\merge_dict.pkl
停用词输出: ../data_processed\new_stopwords.pkl
固化文本输出: ../data_processed\china_news_solidified.pkl
将使用 8 个进程进行并行处理。


### **块 2: 从决策文件生成规则词典**

**目标:** 读取经过人工审核的两个CSV文件 (`expert_rules.csv` 和 `candidate_phrases_for_review_reviewed.csv`)，并根据“专家优先”的原则，生成最终的合并词典 (`merge_dict`) 和自定义停用词集合 (`new_stopwords`)。

In [2]:
# =============================================================================
# --- 块 2: 从决策文件生成规则词典 ---
# =============================================================================

print("--- 阶段 4.1: 开始生成规则词典与停用词列表 ---")
start_time = time.time()

# 作用: 初始化用于存储合并规则的字典和存储自定义停用词的集合。
merge_dict = {}
new_stopwords = set()

try:
    # 作用: 首先加载并处理专家规则文件（最高优先级）。
    if os.path.exists(EXPERT_RULES_PATH):
        df_expert = pd.read_csv(EXPERT_RULES_PATH)
        for _, row in df_expert.iterrows():
            phrase = str(row['phrase_to_merge']).strip().lower()
            standard = str(row['standard_form']).strip()
            if phrase and standard:
                merge_dict[phrase] = standard
        print(f"✅ 从 {EXPERT_RULES_PATH} 加载了 {len(df_expert)} 条专家规则。")
    else:
        print(f"ℹ️ 未找到专家规则文件: {EXPERT_RULES_PATH}，跳过。")

    # 作用: 接着加载并处理经过人工审核的候选短语文件。
    df_reviewed = pd.read_csv(REVIEWED_CANDIDATES_PATH)
    print(f"✅ 从 {REVIEWED_CANDIDATES_PATH} 加载了 {len(df_reviewed)} 条已审核的候选。")

    # 作用: 对'action_code'列进行清洗，确保其为整数类型，以便进行判断。
    df_reviewed['action_code'] = pd.to_numeric(df_reviewed['action_code'], errors='coerce')
    df_reviewed.dropna(subset=['action_code'], inplace=True)
    df_reviewed['action_code'] = df_reviewed['action_code'].astype(int)

    reviewed_merges = 0
    reviewed_stopwords = 0

    # 作用: 遍历审核后的文件，根据action_code将规则分配到合并字典或停用词集合。
    for _, row in df_reviewed.iterrows():
        phrase = str(row['candidate_phrase']).strip().lower()
        if not phrase: continue

        if row['action_code'] == 1: # 值为1表示合并
            standard = str(row['standard_form']).strip()
            # 确保专家规则优先，如果该短语已存在于字典中，则不覆盖。
            if phrase and standard and phrase not in merge_dict:
                merge_dict[phrase] = standard
                reviewed_merges += 1
        elif row['action_code'] == 2: # 值为2表示设为停用词
            new_stopwords.update(phrase.split())
            reviewed_stopwords += 1

    print(f"  - 从审核文件中处理了 {reviewed_merges} 条新的合并规则。")
    print(f"  - 从审核文件中处理了 {reviewed_stopwords} 条新的停用词短语。")
    print("-" * 30)
    print(f"📊 最终规则库统计:")
    print(f"   - 总合并规则数 (merge_dict): {len(merge_dict)}")
    print(f"   - 总新增停用词数 (new_stopwords): {len(new_stopwords)}")
    print("-" * 30)

    # 作用: 将最终生成的规则对象保存到磁盘，以便后续流程使用。
    with open(MERGE_DICT_PATH, 'wb') as f:
        pickle.dump(merge_dict, f)
    print(f"✅ [机器用] 合并词典已保存到: {MERGE_DICT_PATH}")

    with open(NEW_STOPWORDS_PATH, 'wb') as f:
        pickle.dump(new_stopwords, f)
    print(f"✅ [机器用] 新增停用词列表已保存到: {NEW_STOPWORDS_PATH}")

    # 作用: 将合并字典保存为CSV格式，用于人工检查和存档。
    MERGE_DICT_CSV_PATH = os.path.join(BASE_DATA_PROCESSED_PATH, 'merge_dict_for_review.csv')
    df_merge_dict = pd.DataFrame(list(merge_dict.items()), columns=['phrase_to_merge', 'standard_form'])
    df_merge_dict.to_csv(MERGE_DICT_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"✅ [人类用] 合并词典已保存到CSV: {MERGE_DICT_CSV_PATH}")

    print(f"规则生成耗时: {time.time() - start_time:.2f} 秒。")

except FileNotFoundError as e:
    print(f"❌ 错误: 缺少必要的输入文件: {e.filename}。")
    print("请确保已完成人工审核并正确放置文件。")
except Exception as e:
    print(f"❌ 处理规则文件时发生未知错误: {e}")

--- 阶段 4.1: 开始生成规则词典与停用词列表 ---
ℹ️ 未找到专家规则文件: ../data_processed\expert_rules.csv，跳过。
✅ 从 ../data_processed\candidate_phrases_for_review_reviewed.csv 加载了 3140 条已审核的候选。
  - 从审核文件中处理了 1826 条新的合并规则。
  - 从审核文件中处理了 846 条新的停用词短语。
------------------------------
📊 最终规则库统计:
   - 总合并规则数 (merge_dict): 1826
   - 总新增停用词数 (new_stopwords): 736
------------------------------
✅ [机器用] 合并词典已保存到: ../data_processed\merge_dict.pkl
✅ [机器用] 新增停用词列表已保存到: ../data_processed\new_stopwords.pkl
✅ [人类用] 合并词典已保存到CSV: ../data_processed\merge_dict_for_review.csv
规则生成耗时: 0.12 秒。


### **块 3: 构建高效替换引擎**

**目标:** 利用 `pyahocorasick` 库，将上一步生成的、可能非常庞大的 `merge_dict` 编译成一个Aho-Corasick自动机。这个自动机是实现大规模、高性能文本替换的关键，其查找效率远超常规的循环或正则表达式方法。

In [3]:
# =============================================================================
# --- 块 3: 构建高效替换引擎 ---
# =============================================================================

print("\n--- 阶段 4.2a: 构建 Aho-Corasick 高效替换引擎 ---")

automaton = None

# 作用: 将合并字典编译成一个Aho-Corasick自动机，用于后续大规模、高性能的多模式字符串匹配和替换。
if 'merge_dict' in locals() and merge_dict:
    automaton = ahocorasick.Automaton()

    # 作用: 将每个待查找的短语及其对应的(长度, 标准形式)存入自动机。
    # 存入长度是为了在匹配时直接获取，避免重复计算，提升效率。
    for phrase, standard_form in tqdm(merge_dict.items(), desc="构建自动机"):
        automaton.add_word(phrase, (len(phrase), standard_form))

    # 作用: 完成自动机的构建，使其进入可匹配状态。
    automaton.make_automaton()
    print(f"✅ Aho-Corasick 自动机构建完成，包含 {len(merge_dict)} 条规则。")
else:
    print("⚠️ 警告: 合并词典 (merge_dict) 为空或未定义，无法构建替换引擎。")


--- 阶段 4.2a: 构建 Aho-Corasick 高效替换引擎 ---


构建自动机:   0%|          | 0/1826 [00:00<?, ?it/s]

✅ Aho-Corasick 自动机构建完成，包含 1826 条规则。


### **块 4: 应用规则固化文本**

**目标:** 加载完整的中国新闻语料库，并应用上一步构建的自动机，对每一篇文章的`CONTENT`进行实体和短语的“固化”替换。这是整个流程的核心执行步骤。

In [4]:
# =============================================================================
# --- 块 4: 应用规则固化文本 (带全方位验证监控) ---
# =============================================================================

print("\n--- 阶段 4.2b: 开始单线程文本固化处理 ---")
start_time_solidify = time.time()

# 作用: 初始化函数内部的监控计数器。
def initialize_counters():
    solidify_text_definitively.total_matches = 0
    solidify_text_definitively.processed_with_matches = 0

def solidify_text_definitively(text, automaton_obj):
    """
    作用：对单个文本字符串应用Aho-Corasick自动机进行实体固化。
    """
    if not isinstance(text, str) or not automaton_obj:
        return ""

    text_lower = text.lower()
    parts = []
    last_end = 0

    all_matches = []
    for end_index, (phrase_len, standard_form) in automaton_obj.iter(text_lower):
        start_index = end_index - phrase_len + 1
        is_start_boundary = (start_index == 0) or (not text_lower[start_index - 1].isalnum())
        is_end_boundary = (end_index + 1 == len(text_lower)) or (not text_lower[end_index + 1].isalnum())

        if is_start_boundary and is_end_boundary:
            all_matches.append((start_index, end_index, standard_form))

    # 作用: [监控点] 如果找到了任何匹配项，则更新计数器。
    if len(all_matches) > 0:
        solidify_text_definitively.total_matches += len(all_matches)
        solidify_text_definitively.processed_with_matches += 1

    if not all_matches:
        return text

    final_matches = []
    i = 0
    while i < len(all_matches):
        current_best_match = all_matches[i]
        j = i + 1
        while j < len(all_matches) and all_matches[j][0] <= current_best_match[1]:
            if (all_matches[j][1] - all_matches[j][0]) > (current_best_match[1] - current_best_match[0]):
                current_best_match = all_matches[j]
            j += 1

        final_matches.append(current_best_match)
        i = j

    for start_index, end_index, standard_form in final_matches:
        if start_index >= last_end:
            parts.append(text[last_end:start_index])
            parts.append(standard_form)
            last_end = end_index + 1

    parts.append(text[last_end:])

    return "".join(parts)


try:
    # 作用：加载筛选后的新闻数据。
    if not os.path.exists(FINAL_CHINA_NEWS_PATH):
         raise FileNotFoundError(f"新闻语料文件未找到: {FINAL_CHINA_NEWS_PATH}")

    read_nrows = TEST_SAMPLE_SIZE if TEST_MODE else None
    print("正在加载新闻数据...")
    df = pd.read_csv(FINAL_CHINA_NEWS_PATH, nrows=read_nrows)

    # 作用: [验证点] 验证加载的数据量和基本信息。
    print(f"✅ 成功加载 {len(df)} 篇新闻进行处理。")
    if not df.empty:
        print(f"📅 数据时间范围: {df['DATE'].min()} 到 {df['DATE'].max()}")
        print(f"📊 数据列信息: {list(df.columns)}")

    # 作用：确保'TITLE'和'CONTENT'列为字符串类型，并将NaN值转换为空字符串。
    df['TITLE'] = df['TITLE'].astype(str).fillna('')
    df['CONTENT'] = df['CONTENT'].astype(str).fillna('')

    # 作用：应用实体固化规则。
    if 'automaton' in locals() and automaton:
        print("\n开始文本固化处理...")

        # 作用: [验证点] 打印计划处理的文本总数。
        total_texts = len(df)
        print(f"📈 计划处理文本数量: {total_texts}")

        # 作用: 在处理前重置/初始化函数内的监控计数器。
        initialize_counters()

        # 作用：将TITLE和CONTENT在内存中动态合并成一个文本流，并应用固化函数。
        text_iterator = (title + ' . ' + content for title, content in zip(df['TITLE'], df['CONTENT']))

        tqdm.pandas(desc="固化文本")
        df['content_solidified'] = pd.Series(text_iterator, index=df.index).progress_apply(
            lambda text: solidify_text_definitively(text, automaton)
        )

        print(f"✅ 文本固化完成！耗时: {(time.time() - start_time_solidify) / 60:.2f} 分钟。")

        # 作用: [验证点] 打印详尽的处理结果统计报告。
        processed_count = df['content_solidified'].count()
        print(f"\n--- 处理结果验证 ---")
        print(f"✅ 实际处理文本数量: {processed_count}")
        if processed_count == total_texts:
            print("✅ 完整性检查通过：所有文本行均已成功处理。")
        else:
            print(f"⚠️ 完整性检查失败：处理数量不匹配！原始: {total_texts}, 处理后非空: {processed_count}")

        original_merged_text = df['TITLE'] + ' . ' + df['CONTENT']
        changed_count = (df['content_solidified'] != original_merged_text).sum()

        print(f"\n--- 固化效果统计 ---")
        print(f"🔄 文本被修改（固化）的数量: {changed_count} / {total_texts}")
        if total_texts > 0:
            print(f"📊 固化率: {changed_count / total_texts * 100:.2f}%")
        print(f"🔢 函数内部统计: 在 {solidify_text_definitively.processed_with_matches} 篇被修改的文章中，总共执行了 {solidify_text_definitively.total_matches} 次短语匹配。")

        # 作用：随机抽样文章，打印其原始与固化后的内容，用于人工快速验证。
        print("\n--- 抽样检查结果 ---")
        sample_size = min(5, len(df))
        if sample_size > 0:
            pd.set_option('display.max_colwidth', 400)
            for i, row in df.sample(sample_size).iterrows():
                print(f"--- 文章 {i} ---")
                print(f"【原始TITLE】: {str(row['TITLE'])}")
                print(f"【原始CONTENT】: {str(row['CONTENT'])[:300]}...")
                print(f"【固化后】: {str(row['content_solidified'])[:400]}...")
                print("-" * 20)
    else:
        print("\n自动机未构建，跳过文本固化。将直接合并TITLE和CONTENT。")
        df['content_solidified'] = df['TITLE'] + ' . ' + df['CONTENT']

except FileNotFoundError as e:
    print(f"❌ 错误: {e}")
except Exception as e:
    print(f"❌ 固化文本过程中发生未知错误: {e}")
    import traceback
    traceback.print_exc()


--- 阶段 4.2b: 开始单线程文本固化处理 ---
正在加载新闻数据...
✅ 成功加载 180630 篇新闻进行处理。
📅 数据时间范围: 1984-01-02 到 2025-03-15
📊 数据列信息: ['DATE', 'TITLE', 'CONTENT']

开始文本固化处理...
📈 计划处理文本数量: 180630


固化文本:   0%|          | 0/180630 [00:00<?, ?it/s]

✅ 文本固化完成！耗时: 0.35 分钟。

--- 处理结果验证 ---
✅ 实际处理文本数量: 180630
✅ 完整性检查通过：所有文本行均已成功处理。

--- 固化效果统计 ---
🔄 文本被修改（固化）的数量: 173985 / 180630
📊 固化率: 96.32%
🔢 函数内部统计: 在 173985 篇被修改的文章中，总共执行了 2246311 次短语匹配。

--- 抽样检查结果 ---
--- 文章 99837 ---
【原始TITLE】: Are Plastics Making Us Fat?
【原始CONTENT】: Weight-loss crazes are as American as apple pie -- make that Slim Fast shakes. But despite our countless diet fads, the obesity rate has more than doubled in the last 30 years. Perhaps that's because Americans haven't tried "The New American Diet," which promises to reveal "why your weight isn't you...
【固化后】: Are Plastics Making Us Fat? . Weight-loss crazes are as American as apple pie -- make that Slim Fast shakes. But despite our countless diet fads, the obesity rate has more than doubled in the last 30 years. Perhaps that's because Americans haven't tried "The New American Diet," which promises to reveal "why your weight isn't your fault" and reverse "the obesogen effect." Haven't heard of the "obes...
---------

### **块 5: 保存最终结果**

**目标:** 将包含新增的`content_solidified`列的DataFrame完整保存到磁盘。推荐使用Pickle格式，因为它能无损、高效地存储Pandas对象，为后续的深度清洗步骤（`04_...`）提供一个干净的起点。

In [5]:
# =============================================================================
# --- 块 5: 保存最终结果 ---
# =============================================================================

# 作用: 负责对固化后的结果进行检查，并保存为最终的Pickle和CSV文件。
if 'df' in locals() and 'content_solidified' in df.columns:
    print("\n--- 阶段 4.2c: 保存固化后的结果 ---")
    try:
        # 作用: 定义两种格式的输出路径。
        SOLIDIFIED_TEXT_PKL_PATH = os.path.join(BASE_DATA_PROCESSED_PATH, 'china_news_solidified.pkl')
        SOLIDIFIED_TEXT_CSV_PATH = os.path.join(BASE_DATA_PROCESSED_PATH, 'china_news_solidified_for_review.csv')

        # 作用: 保存为Pickle格式，用于后续Python脚本高效读取。
        df.to_pickle(SOLIDIFIED_TEXT_PKL_PATH)
        print(f"✅ [机器用] 包含固化文本的DataFrame已保存到Pickle: {SOLIDIFIED_TEXT_PKL_PATH}")

        # 作用: 保存为CSV格式，用于人工查阅和审计。
        df.to_csv(SOLIDIFIED_TEXT_CSV_PATH, index=False, encoding='utf-8-sig')
        print(f"✅ [人类用] 包含固化文本的DataFrame已保存到CSV: {SOLIDIFIED_TEXT_CSV_PATH}")

        # 作用: 在服务器环境下，将产出文件从临时目录同步回项目的数据目录。
        if RUNNING_ENV == 'dsw':
            print(f"DSW模式：正在将结果同步回项目目录...")
            shutil.copy(SOLIDIFIED_TEXT_PKL_PATH, SOLIDIFIED_TEXT_ORIGINAL)
            shutil.copy(SOLIDIFIED_TEXT_CSV_PATH,
                        os.path.join(BASE_DATA_PROCESSED_PATH, 'china_news_solidified_for_review.csv'))
            print("同步完成。")

        # 作用: 打印清晰的总结和下一步操作指引。
        print(f"\n✅✅✅ “实体固化”流程全部完成！ ✅✅✅")
        print(f"产出文件:")
        print(f"  - {SOLIDIFIED_TEXT_PKL_PATH}")
        print(f"  - {SOLIDIFIED_TEXT_CSV_PATH}")
        print("\n下一步是运行 '03_Deep_Text_Cleaning.ipynb'，对'content_solidified'列进行最终的标准化和噪声移除。")

    except Exception as e:
        print(f"❌ 保存文件时发生错误: {e}")
else:
    print("没有可供保存的固化文本数据。")


--- 阶段 4.2c: 保存固化后的结果 ---
✅ [机器用] 包含固化文本的DataFrame已保存到Pickle: ../data_processed\china_news_solidified.pkl
✅ [人类用] 包含固化文本的DataFrame已保存到CSV: ../data_processed\china_news_solidified_for_review.csv

✅✅✅ “实体固化”流程全部完成！ ✅✅✅
产出文件:
  - ../data_processed\china_news_solidified.pkl
  - ../data_processed\china_news_solidified_for_review.csv

下一步是运行 '03_Deep_Text_Cleaning.ipynb'，对'content_solidified'列进行最终的标准化和噪声移除。
